# A Post Office – A Real Life Story

Let us begin with an everyday story: there is a small post office with one clerk serving the arriving customers. Customers have differing wishes leading to different serving times, from ```1 - 5 minutes```. We have to add a little variation to serving times counting for variation in customer habits and clerk performance. The arrival rate of customers is about 18 per hour, every ```3.33 minutes``` or ```3 minutes, 20 seconds``` on average. Our post office is small and customer patience is limited, so queue length is limited to 5 customers. 

We have provided 10% extra capacity, so our expectation is that there should not be too many customers discouraged for long waiting times or for full queues.

Let's do a simulation in [```Julia```](https://julialang.org) using [```SimJulia```](https://github.com/BenLauwens/SimJulia.jl). We need 

1. a source: all the people, providing an unlimited supply for customers,
2. customers with their demands and their limited patience,
3. a queue and
4. our good old clerk.

In [1]:
using SimJulia, Distributions

@resumable function people(sim::Simulation, β::Float64,
                           queue::Array{Int64,1}, clerk::Resource)
    i = 1
    while true
        Δt = rand(Exponential(β))
        @yield return Timeout(sim, Δt)
        @coroutine customer(sim, i, queue, clerk)
        i += 1
    end
end

@resumable function customer(sim::Simulation, n::Int64,
                             queue::Array{Int64,1}, clerk::Resource)
    if length(queue) > 5
        println(@sprintf("%0.2f: Queue is full! Customer %d leaves", now(sim), n))
        return
    else
        arrivaltime = now(sim)
        println(@sprintf("%0.2f: Customer %d enqueues", now(sim), n))
        push!(queue, n)
        @yield return Request(clerk)
        shift!(queue)
        println(@sprintf("%0.2f: Customer %d being served", now(sim), n))
        Δt = rand(DiscreteUniform(1, 5)) + randn()*0.2
        @yield return Timeout(sim, Δt)
        @yield return Release(clerk)
        println(@sprintf("%0.2f: Customer %d leaves after %0.2f min", now(sim), n, now(sim)-arrivaltime))
    end
end

srand(1234)       # seed random number generator for reproducibility
queue = Int64[]
sim = Simulation()
clerk = Resource(sim, 1)
@coroutine people(sim, 3.333, queue, clerk)
run(sim, 600)    # run for 600 minutes
println(queue, " yet in queue")


8.28: Customer 1 enqueues
8.28: Customer 1 being served
13.10: Customer 1 leaves after 4.82 min
13.33: Customer 2 enqueues
13.33: Customer 2 being served
17.44: Customer 2 leaves after 4.11 min
17.69: Customer 3 enqueues
17.69: Customer 3 being served
18.84: Customer 4 enqueues
19.58: Customer 3 leaves after 1.90 min
19.58: Customer 4 being served
21.10: Customer 5 enqueues
21.61: Customer 4 leaves after 2.77 min
21.61: Customer 5 being served
24.63: Customer 5 leaves after 3.53 min
25.66: Customer 6 enqueues
25.66: Customer 6 being served
26.72: Customer 7 enqueues
27.68: Customer 6 leaves after 2.01 min
27.68: Customer 7 being served
28.40: Customer 7 leaves after 1.67 min
29.60: Customer 8 enqueues
29.60: Customer 8 being served
33.96: Customer 8 leaves after 4.36 min
35.70: Customer 9 enqueues
35.70: Customer 9 being served
35.94: Customer 10 enqueues
37.13: Customer 11 enqueues
37.73: Customer 12 enqueues
37.85: Customer 9 leaves after 2.15 min
37.85: Customer 10 being served
40.9

384.39: Customer 99 leaves after 13.72 min
384.39: Customer 100 being served
385.60: Customer 100 leaves after 14.86 min
385.60: Customer 101 being served
386.61: Customer 107 enqueues
390.45: Customer 101 leaves after 17.33 min
390.45: Customer 102 being served
391.46: Customer 102 leaves after 16.14 min
391.46: Customer 103 being served
393.70: Customer 108 enqueues
393.74: Customer 109 enqueues
396.43: Customer 103 leaves after 18.61 min
396.43: Customer 104 being served
399.49: Customer 104 leaves after 20.86 min
399.49: Customer 106 being served
400.17: Customer 110 enqueues
400.54: Customer 111 enqueues
400.95: Customer 112 enqueues
403.46: Customer 106 leaves after 20.84 min
403.46: Customer 107 being served
403.57: Customer 113 enqueues
406.15: Queue is full! Customer 114 leaves
408.18: Customer 107 leaves after 21.57 min
408.18: Customer 108 being served
408.62: Customer 115 enqueues
409.39: Queue is full! Customer 116 leaves
411.18: Queue is full! Customer 117 leaves
411.52: 

We did run the simulation 600 minutes. 172 customers came, 160 customers were served, 10 left beforehand because the queue was full, 2 were still in queue when simulation was over. Many customers had waiting times of meor than 10, 15 or even 20 minutes.

So many customers will remain angry. If this is the situation all days, our post office will have an evil reputation. What should we do?

## Conclusion

What we see here are the **effects of variation** in arrivals, in demands and in serving time on system performance. In this case 10% extra capacity is not enough to provide for variation and to serve the customers well.

This is what this Repository ```PFlow``` is all about: to get systems flowing and to see what may happen beforehand before the system breaks down.